In [59]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
DATA_PATH = "./data/train.json"

with open(DATA_PATH, "r") as fichier:
    data = json.load(fichier)

def to_document(data):
    res = {}
    for key in data:
        s = ""
        for sentence in data[key]:
            s += sentence + " "
        res[key] = s
    return res

dico = to_document(data)
df = pd.DataFrame(dico.items(), columns=['Label', 'Text'])
df

,Label,Text
0,Politics,The mayor announced a new initiative to improv...
1,Health,Regular exercise and a balanced diet are key t...
2,Finance,The stock market saw a significant drop follow...
3,Travel,Visiting the Grand Canyon is a breathtaking ex...
4,Food,The new restaurant in town offers a fusion of ...
5,Education,The school district is implementing a new curr...
6,Environment,Climate change is causing a significant rise i...
7,Fashion,The new fashion trend is all about sustainabil...
8,Science,NASA's Mars Rover has made significant discove...
9,Sports,The NBA Finals are set to begin next week with...


In [61]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Text'])

In [62]:
file_path = "./data/test_shuffle.txt"

with open(file_path, "r") as file:
    text = file.read()

result = pd.DataFrame(columns=["ID", "Sentence"])

for i, sentence in enumerate(text.split("\n")):
    result = pd.concat([result, pd.DataFrame([[i, sentence]], columns=["ID", "Sentence"])], ignore_index=True)

result["Label"] = result["Sentence"].map(lambda x: df.iloc[cosine_similarity(X, vectorizer.transform([x])).argmax(), 0])

In [63]:
result = result.iloc[:-1]
result.loc[:, ["ID", "Label"]].to_csv("./result.csv", index=False)